In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re



In [58]:

# Đọc dữ liệu từ tập CSV vào DataFrame
df = pd.read_csv('DATADULICHQUYNHON.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df


,Name_Hotel,original_price_list,Link,star_rating_hotel_list,Location,Convenient,Aria_locations,Comment_Hotel,User_vote_hotel,Review_Hotel,hightlight_Hotel
0,Grand Hyams Hotel - Quy Nhon Beach,VND 3.950.100,https://www.booking.com/hotel/vn/grand-hyams-q...,5.0,"28 Nguyen Hue, Le Loi ward, Quy Nhon city, Bin...","Hồ bơi trong nhà, WiFi miễn phí, Phòng gia ...","Tượng Đài Chiến Thắng: 150 m, Công Viên Bàu Se...","Tram(Việt Nam): “Sang trọng, yên tĩnh, các bạ...",9.3,"Nằm ở Quy Nhơn, cách Quy Nhon Beach 2 phút đi ...","Nhìn ra biển,Nhìn ra địa danh nổi tiếng,Hồ bơi..."
1,FLC City Hotel Beach Quy Nhon,VND 3.937.950,https://www.booking.com/hotel/vn/flc-beach-cit...,5.0,"An Duong Vuong Road Quy Nhon City, Binh Dinh P...","Hồ bơi trong nhà, Giáp biển, WiFi miễn phí, ...","Công Viên: 300 m, Đu Quay Uốn Lượn: 400 m, Côn...","Huy(Việt Nam): “Phòng rộng, view đẹp, vị trí ...",8.6,"Nằm giáp biển, FLC City Hotel Beach Quy Nhon c...","Nhìn ra biển, Địa điểm hàng đầu: Được khách gầ..."
2,RAON Hotel - STAY 24H,VND 1.426.513,https://www.booking.com/hotel/vn/nam-thu.vi.ht...,4.0,"09 Nguyễn Thị Định, Quy Nhơn, Việt Nam – Vị t...","Hồ bơi ngoài trời, Giáp biển, WiFi miễn phí,...","Công Viên: 500 m, Công Viên: 600 m, Đu Quay Uố...","Gabriel(Việt Nam): “Mọi thứ đều ổn, sạch sẽ, ...",9.3,"Tọa lạc ở Quy Nhơn, cách Quy Nhon Beach 7 phút...","Hồ bơi có tầm nhìn,Hướng nhìn ra đường phố yên..."
3,MENTO HOTEL QUY NHƠN,VND 1.863.000,https://www.booking.com/hotel/vn/mento-quy-nho...,3.0,"Nguyễn Huệ 409C - 409D, Quy Nhơn, Việt Nam – ...","3 hồ bơi, Giáp biển, WiFi miễn phí, Xe đưa đ...","Ngã 6 Ngô Mây: 250 m, Công Viên: 400 m, Đu Qua...",No comment,8.4,"Nằm giáp biển, MENTO HOTEL QUY NHƠN cung cấp c...","Nhìn ra thành phố,Nhìn ra núi,Nhìn ra địa danh..."
4,Moon's House Homestay Quy Nhơn,VND 1.050.000,https://www.booking.com/hotel/vn/moon-39-s-hou...,2.0,"82 Vũ Bảo, Quy Nhơn, Việt Nam – Vị trí xuất s...","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Ngã 6 Ngô Mây: 350 m, Công Viên: 550 m, Đu Qua...","Vy(Việt Nam): “Không gian vừa đủ, sạch sẽ, mề...",9.5,Nằm ở Quy Nhơn và cách Quy Nhon Beach chưa đến...,"Sân hiên, Địa điểm hàng đầu: Được khách gần đâ..."
...,...,...,...,...,...,...,...,...,...,...,...
438,Q House- Green Villa Quy Nhơn,VND 8.967.240,https://www.booking.com/hotel/vn/q-house-homes...,0.0,"70/14/12 70/14/13 Han Mac Tu, Ghenh Rang Ward,...","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Công Viên: 600 m, Công Viên: 1,1 km, Đu Quay U...",No comment,9.0,"Cách Quy Nhon Beach 4 phút đi bộ, Q House- Gre...","Nhìn ra thành phố,Sân hiên,Nhìn ra vườn,Hướng ..."
439,Căn hộ du lịch Quy Nhơn giá rẻ-view biển 7,VND 2.904.000,https://www.booking.com/hotel/vn/quy-nhon-colo...,0.0,"Chung cư An Phú Thịnh Garden Tower, Khu đô thị...","WiFi miễn phí, Chỗ đỗ xe miễn phí","Công Viên Đống Đa: 2,7 km, Công Viên Đống Đa: ...",No comment,8.4,"Tọa lạc ở Quy Nhơn thuộc Bình Định, Căn hộ du ...",Sân hiênNo hightlight
440,Quy Nhon Homestay BnB,VND 8.997.000,https://www.booking.com/hotel/vn/quynhonhomest...,0.0,"20 Trần Kỳ Phong, Quy Nhơn, Bình Định, Quy Nhơ...","Giáp biển, WiFi miễn phí, Xe đưa đón sân ba...","Công Viên: 500 m, Công Viên: 900 m, Đu Quay Uố...",Hồng(Việt Nam): “Hàng xóm thân thiện! Địa điể...,8.4,Nằm ở Quy Nhơn và cách Quy Nhon Beach 6 phút đ...,"Nhìn ra thành phố,Nhìn ra núi,Sân hiên,Hướng n..."
441,CĂN HỘ QUY NHƠN,VND 5.400.000,https://www.booking.com/hotel/vn/can-ho-quy-nh...,0.0,"76 Trần Hưng Đạo 26, Quy Nhơn, Việt Nam – Hiể...","Hồ bơi, WiFi miễn phí, Chỗ đỗ xe miễn phí","Tượng Đài Chiến Thắng: 750 m, Công Viên Đống Đ...",No comment,8.4,No review,Sân hiênNo hightlight


In [59]:
df.columns

Index(['Name_Hotel', 'original_price_list', 'Link', 'star_rating_hotel_list',
       'Location', 'Convenient', 'Aria_locations', 'Comment_Hotel',
       'User_vote_hotel', 'Review_Hotel', 'hightlight_Hotel'],
      dtype='object')

In [61]:
df['original_price_list'] = pd.to_numeric(df['original_price_list'], errors='coerce')
df['numeric_values'] = df['numeric_values'].apply(lambda x: [float(value) for value in x])
df.fillna(0, inplace=True)  # Hoặc thay thế NaN bằng giá trị khác phù hợp


In [74]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
import re

# Thêm cột mô tả cho PCA
df['description'] = (
    df['star_rating_hotel_list'].astype(str) +
    ' ' +
    df['original_price_list'].astype(str) +
    ' ' +
    df['User_vote_hotel'].astype(str)
)

# Trích xuất giá trị số từ cột mô tả bằng biểu thức chính quy
df['numeric_values'] = df['description'].apply(lambda x: [float(value.replace(',', '')) for value in re.findall(r'\d+\.\d+|\d+', x)])

# Xử lý giá trị NaN trong cột numeric_values
# Xử lý giá trị NaN bằng giá trị trung bình của cột
df['numeric_values'] = df['numeric_values'].apply(lambda x: x if not pd.isna(x).any() else df['numeric_values'].mean())


# Chuyển đổi cột numeric_values thành một mảng 2D
X_content = pd.DataFrame(df['numeric_values'].tolist()).to_numpy()

# Chuẩn hóa dữ liệu
X_normalized_content = StandardScaler().fit_transform(X_content)

# Áp dụng PCA cho các đặc trưng dựa trên nội dung
pca_content = PCA(n_components=2)
pca_result_content = pca_content.fit_transform(X_normalized_content)
pca_df = pd.DataFrame(pca_result_content, columns=['PC1_content', 'PC2_content'])

# Kết hợp kết quả PCA với kết quả PCA trước đó (nếu có)
combined_pca_result = pd.concat([pd.DataFrame(pca_result_content, columns=['PC1_content', 'PC2_content']), pca_df], axis=1)

# Tính toán độ tương đồng cosine trên kết quả PCA kết hợp
cosine_similarities_combined = linear_kernel(combined_pca_result, combined_pca_result)

# Hàm đề xuất dựa trên độ tương đồng cosine
def combined_recommendation(hotel_name, cosine_similarities, df):
    hotel_index = df[df['Name_Hotel'] == hotel_name].index
    if not hotel_index.empty:
        hotel_index = hotel_index[0]
        similar_hotels = list(enumerate(cosine_similarities[hotel_index]))
        similar_hotels = sorted(similar_hotels, key=lambda x: x[1], reverse=True)
        similar_hotels = similar_hotels[1:11]  # Lấy 10 khách sạn tương tự

        # Lấy tên của 10 khách sạn tương tự
        recommended_hotels = [df.iloc[hotel[0]]['Name_Hotel'] for hotel in similar_hotels]
        return recommended_hotels
    else:
        print(f'Không tìm thấy khách sạn: {hotel_name}')
        return []

# Đề xuất 10 khách sạn tương tự với khách sạn có tên là 'Tên khách sạn cần đề xuất'
recommended_hotels_combined = combined_recommendation("Q House- Green Villa Quy Nhơn", cosine_similarities_combined, df)
print(recommended_hotels_combined)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [65]:
df['description']

0      5.0 Tượng Đài Chiến Thắng: 150 m, Công Viên Bà...
1      5.0 Công Viên: 300 m, Đu Quay Uốn Lượn: 400 m,...
2      4.0 Công Viên: 500 m, Công Viên: 600 m, Đu Qua...
3      3.0 Ngã 6 Ngô Mây: 250 m, Công Viên: 400 m, Đu...
4      2.0 Ngã 6 Ngô Mây: 350 m, Công Viên: 550 m, Đu...
                             ...                        
438    0.0 Công Viên: 600 m, Công Viên: 1,1 km, Đu Qu...
439    0.0 Công Viên Đống Đa: 2,7 km, Công Viên Đống ...
440    0.0 Công Viên: 500 m, Công Viên: 900 m, Đu Qua...
441    0.0 Tượng Đài Chiến Thắng: 750 m, Công Viên Đố...
442    5.0 Tượng Đài Chiến Thắng: 750 m, Công Viên Đố...
Name: description, Length: 443, dtype: object

In [64]:
# Xử lý dữ liệu, nếu cần
# Ví dụ: Kết hợp các trường 'Tên khách sạn' và 'Tiện ích' thành một trường duy nhất 'Mô tả'
#df['Mô tả'] = df['Name_Hotel'] + ' ' + df['star_rating_hotel_list'] + ' ' + df['voting_user_list'] + ' ' + df['review_list']
df['description'] = (
    df['star_rating_hotel_list'].astype(str) +
    ' ' +
    df['Aria_locations'].astype(str) +
    ' ' +
    df['Convenient'].astype(str) +
    ' ' +
    df['Review_Hotel'].astype(str) +
    ' ' +
    df['hightlight_Hotel'].astype(str) +
    ' ' +
    df['User_vote_hotel'].astype(str)
)

# Sử dụng TF-IDF để biểu diễn văn bản thành ma trận số
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'].fillna(''))

# Tính cosine similarity giữa các khách sạn
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Hàm đề xuất dựa trên cosine similarity
def content_based_recommendation(hotel_name, cosine_similarities, df):
    hotel_index = df[df['Name_Hotel'] == hotel_name].index
    if not hotel_index.empty:
        hotel_index = hotel_index[0]
        similar_hotels = list(enumerate(cosine_similarities[hotel_index]))
        similar_hotels = sorted(similar_hotels, key=lambda x: x[1], reverse=True)
        similar_hotels = similar_hotels[1:11]  # Lấy 10 khách sạn tương tự

        # Lấy tất cả giá trị từ danh sách similar_hotels
        recommended_hotels = [df.iloc[hotel[0]]['Name_Hotel'] for hotel in similar_hotels]
        return recommended_hotels
    else:
        print(f'Hotel not found: {hotel_name}')
        return []


# Đề xuất 10 khách sạn tương tự với khách sạn có tên là 'Tên khách sạn cần đề xuất'
recommended_hotels = content_based_recommendation("Q House- Green Villa Quy Nhơn", cosine_similarities, df)
print(recommended_hotels)

['TMS View Trực Diện Biển', 'Green Hotel Quy Nhơn', 'Q House- FLC Sea Tower', 'FLC Sea Tower Quy Nhon- Q House Codotel', 'TMS Quy Nhơn - RUBY Luxury Apartment', 'TMS Residences Beachfront Quy Nhon', 'Home Thiên Trang 3', 'ND Travel - FLC Sea Tower Quy Nhơn', 'Quy Nhơn Oceanview Apartmet', 'Bluewaves Apartment Quy Nhon']


In [92]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Hàm đánh giá hiệu suất hệ thống đề xuất
def evaluate_recommendation_system(cosine_similarities, test_data, df):
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for _, row in test_data.iterrows():
        hotel_name = row['Name_Hotel']
        actual_recommendations = row['actual_recommendations'].split(', ')
        
        # Lấy đề xuất từ hệ thống sử dụng PCA
        recommended_hotels_combined = combined_recommendation(hotel_name, cosine_similarities_combined, df)

        # Lấy đề xuất từ hệ thống không sử dụng PCA
        recommended_hotels = content_based_recommendation(hotel_name, cosine_similarities, df)

        # Tính các điểm đánh giá hiệu suất
        precision_scores_combined = precision_score(actual_recommendations, recommended_hotels_combined, average='micro')
        recall_scores_combined = recall_score(actual_recommendations, recommended_hotels_combined, average='micro')
        f1_scores_combined = f1_score(actual_recommendations, recommended_hotels_combined, average='micro')

        precision_scores_no_pca = precision_score(actual_recommendations, recommended_hotels, average='micro')
        recall_scores_no_pca = recall_score(actual_recommendations, recommended_hotels, average='micro')
        f1_scores_no_pca = f1_score(actual_recommendations, recommended_hotels, average='micro')

        # Lưu kết quả
        precision_scores.append({'combined': precision_scores_combined, 'no_pca': precision_scores_no_pca})
        recall_scores.append({'combined': recall_scores_combined, 'no_pca': recall_scores_no_pca})
        f1_scores.append({'combined': f1_scores_combined, 'no_pca': f1_scores_no_pca})

    # Tính trung bình các điểm đánh giá hiệu suất
    avg_precision_combined = sum([score['combined'] for score in precision_scores]) / len(precision_scores)
    avg_recall_combined = sum([score['combined'] for score in recall_scores]) / len(recall_scores)
    avg_f1_combined = sum([score['combined'] for score in f1_scores]) / len(f1_scores)

    avg_precision_no_pca = sum([score['no_pca'] for score in precision_scores]) / len(precision_scores)
    avg_recall_no_pca = sum([score['no_pca'] for score in recall_scores]) / len(recall_scores)
    avg_f1_no_pca = sum([score['no_pca'] for score in f1_scores]) / len(f1_scores)

    print('Evaluation for PCA:')
    print(f'Average Precision: {avg_precision_combined}')
    print(f'Average Recall: {avg_recall_combined}')
    print(f'Average F1 Score: {avg_f1_combined}')

    print('\nEvaluation without PCA:')
    print(f'Average Precision: {avg_precision_no_pca}')
    print(f'Average Recall: {avg_recall_no_pca}')
    print(f'Average F1 Score: {avg_f1_no_pca}')

# Tạo một số dữ liệu mẫu cho tập kiểm tra
test_data = pd.DataFrame({
    'Name_Hotel': ['Hotel_A', 'Hotel_B', 'Hotel_C'],
    'actual_recommendations': ['Hotel_D, Hotel_E, Hotel_F', 'Hotel_A, Hotel_C, Hotel_G', 'Hotel_B, Hotel_F, Hotel_H']
})

# Đánh giá hiệu suất hệ thống đề xuất cho PCA và không sử dụng PCA
evaluate_recommendation_system(cosine_similarities_combined, test_data, df)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [93]:
print(cosine_similarities)

[[1.         0.58797706 0.38877048 ... 0.39274698 0.46429068 0.38254865]
 [0.58797706 1.         0.46317676 ... 0.44465762 0.45032778 0.41688474]
 [0.38877048 0.46317676 1.         ... 0.50588325 0.41595226 0.42393599]
 ...
 [0.39274698 0.44465762 0.50588325 ... 1.         0.40714744 0.47574713]
 [0.46429068 0.45032778 0.41595226 ... 0.40714744 1.         0.36874162]
 [0.38254865 0.41688474 0.42393599 ... 0.47574713 0.36874162 1.        ]]


In [94]:
print(tfidf_matrix)

  (0, 1768)	0.05842898588738355
  (0, 768)	0.043487573482666714
  (0, 2992)	0.042788899482674274
  (0, 2664)	0.06485049101912138
  (0, 2906)	0.0657763171168724
  (0, 9)	0.07905654335755091
  (0, 3092)	0.031455851857236074
  (0, 2920)	0.03065557283218467
  (0, 2875)	0.02794478877936976
  (0, 3088)	0.029283636865981573
  (0, 2195)	0.021645634388997217
  (0, 1968)	0.031455851857236074
  (0, 1706)	0.02083223046888835
  (0, 2045)	0.021733372295892044
  (0, 1526)	0.03787917044479189
  (0, 2736)	0.03434680294934433
  (0, 2779)	0.03542030154929226
  (0, 2346)	0.04481574426606811
  (0, 2868)	0.04512429580364933
  (0, 1144)	0.031381652704988466
  (0, 228)	0.03460939832670214
  (0, 2804)	0.0328449530204994
  (0, 2229)	0.03366423111937039
  (0, 2697)	0.032764984586419466
  (0, 3182)	0.0397862577713224
  :	:
  (499, 2867)	0.03385964456437101
  (499, 108)	0.07380616347322475
  (499, 1244)	0.04561300929969161
  (499, 1203)	0.05888929128677975
  (499, 3076)	0.08043167609300167
  (499, 1555)	0.02870978